In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
# !pip install gurobipy


In [3]:
from gurobipy import *

### Step 1: compute minimum tardy amount t

##### Constants
$\text{Let}$  <br>

$M = \{1, 2, ..., 5\} \text{ be the set of machines,}$<br>

$J = \{1, 2, ..., 12\} \text{ be the set of jobs,}$  <br>

$I = \{1, 2, ..., 6\}\text{ be the set of process number,}$ <br>

$S_{j} = \text{ the splitting timing for job } j \text{, } \forall j \in J \text{, }$ <br>

$D_{j} = \text{ the due time for job } j \text{, } \forall j \in J \text{, }$ <br>

$P = \{boiling, baking, smoking, null\} \text{ be the set of process name,}$ <br>

$ C_{mp} = 
\begin{cases}
    \ 1, & \text{if machine } m \text{ can do process } p \\
    0,     & \text{otherwise}
\end{cases}
\text{, }\forall m \in M, p \in P
$

$A_{ijp} = 
\begin{cases}
    \ 1, & \text{if process } i \text{ in job } j \text{ is } p \\
    0,     & \text{otherwise}
\end{cases}
\text{, }\forall i \in I, j \in J, p \in P\text{, }$

$T_{ij} = \text{time spent by process } i \text{ of job } j \text{ (} T_{ij} = 0 \text{ if } \sum_{p \in P}{A_{ijp}} = 0 \text{) } \forall i \in I, j \in J \text{. }$

$ b= \text{the opening time of the smoking station}$


##### Variables
$\text{def.}$


$ x_{ijm} = 
\begin{cases}
    \ 1, & \text{if the process } i \text{ of job }j \text{ is assign to machine }m\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I, j \in J
$

$w_{j} = 
\begin{cases}
    \ 1, & \text{if job } j \text{ is tardy job }\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I
$

<!-- $ x_{ijm} = 
\begin{cases}
    \ 1, & \text{if the process } i \text{ of job }j \text{ is assign to machine }m\\
     0,     & \text{otherwise}
\end{cases}
\forall i \in I, j \in J
$ -->

$z_{i,j,i',j'} = 
\begin{cases}
    \ 1, & \text{if job } j \text{ of process }i \text{ is scheduled before job }j' \text{ of process } i'\\
     0,     & \text{otherwise}
\end{cases}
\forall i,i' \in I, j,j' \in J
$

$ y_{i,j,i',j',m} =
\begin{cases}
    \ 1, & \text{if } x_{i,j,m} = x_{i',j',m} = 1 \\
     0,     & \text{otherwise}
\end{cases}
\forall i,i' \in I, j,j' \in J, m \in M
$

$ v_{ij} = \text{the completion time of process }i \text{, job }j, \forall i \in I, j \in J
$







$\text{obj. }$ $\min \sum_{j \in J} w_j$ 

$\text{s.t. }$  \
ensure the correctness of order of each process 

**1-a. same job** \
$v_{ij} + T_{ij} - v_{i'j} \leq K \cdot z_{iji'j} \quad \forall i, i' \in I,\: j, j' \in J,\: i' < i,\: j' < j$ \
$v_{i'j} + T_{i'j} - v_{ij} \leq K \cdot (1-z_{iji'j}) \quad \forall i, i' \in I,\: j, j' \in J,\: i' < i,\: j' < j$   

**1-b. same machine** \
$y_{iji'j'm}(v_{ij} \cdot x_{ijm} + T_{ij} - v_{i'j'} \cdot x_{i'j'm}) \leq K \cdot z_{iji'j'} \quad \forall i, i' \in I,\: j, j' \in J,\: m \in M, \: i' < i,\: j' < j$ \
$y_{iji'j'm}(v_{i'j'} \cdot x_{i'j'm} + T_{i'j'} - v_{ij} \cdot x_{ijm}) \leq K \cdot (1 - z_{iji'j'}) \quad \forall i, i' \in I,\: j, j' \in J,\: m \in M, \: i' < i,\: j' < j$

**2. Each process is completed by exactly one machine**

$$\sum_{m \in M}x_{ijm} = 1, \forall i \in I, j \in J$$

**3. Whether a machine can do to process type** \
$x_{ijm} \cdot A_{ijp} \leq C_{mp} \quad \forall p \in P, i \in I, j \in J, m \in M$

**4. Constraint related to "Split"**

$$v_{i+1j}-v_{ij}=T_{i+1j}, \forall i \in I - \{S_{j}, |I|\}, j \in J$$ :      連續

$$x_{i+1jm} = x_{ijm}, \forall i \in I - \{S_{j}, |I|\}, m \in M, j \in J$$:    同一台機器

**5. Minimum Tardiness**\
$v_{|I|j} - (D_j - b) \geq K \cdot w_j \quad \forall j \in J$ 

**6. Non-negative constraints** \
$w_{ij} \in \{0,1\}$


### Step 2: minimize makespan

$\text{obj. } \min \: (\max v_{ij})$ \
$\text{s.t.}$ \
$\sum_{j \in J}w_j = t$  
$\text{and all above constraints.}$ 

In [4]:
import pandas as pd

In [5]:
# data = pd.read_excel('openpyxl","data/OR110-1_case01.xlsx', engine='openpyxl')
df1 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 1')
df2 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 2')
df3 = pd.read_excel ('data/OR110-1_case01.xls', sheet_name='Instance 3')



In [6]:
df2

,Job ID,Process 1,Process 2,Process 3,Process 4,Process 5,Process 6,Splitting Timing,Process 1.1,Process 2.1,Process 3.1,Process 4.1,Process 5.1,Process 6.1,Due Time
0,1,Boiling,Baking,Smoking,NaN,NaN,NaN,1,2.7,1.0,0.5,NaN,NaN,NaN,17:30:00
1,2,Boiling,Baking,Boiling,NaN,NaN,NaN,1,1.6,1.4,0.9,NaN,NaN,NaN,12:30:00
2,3,Boiling,Baking,Boiling,Baking,Boiling,NaN,1,1.0,0.9,0.2,0.2,1.4,NaN,12:30:00
3,4,Smoking,Boiling,Baking,Boiling,Baking,Smoking,4,0.5,0.7,1.0,0.6,0.3,0.5,12:30:00
4,5,Boiling,Baking,Boiling,NaN,NaN,NaN,1,0.8,1.0,0.9,NaN,NaN,NaN,12:30:00
5,6,Boiling,NaN,NaN,NaN,NaN,NaN,0,2.7,NaN,NaN,NaN,NaN,NaN,12:30:00
6,7,Baking,Smoking,NaN,NaN,NaN,NaN,1,1.4,1.5,NaN,NaN,NaN,NaN,12:30:00
7,8,Baking,Smoking,NaN,NaN,NaN,NaN,0,1.1,1.1,NaN,NaN,NaN,NaN,17:30:00
8,9,Boiling,Baking,Boiling,NaN,NaN,NaN,1,0.8,1.0,0.8,NaN,NaN,NaN,17:30:00
9,10,Baking,Boiling,Baking,Boiling,Baking,NaN,3,1.0,0.5,0.7,1.1,1.1,NaN,17:30:00


In [7]:
df = df2
maxProcess = 6
M = 5
J = 11

method = ["Boiling", "Baking", "Smoking"]
P = len(method)

In [8]:

C = {}
machineNumber = 5
C[0,0] = 1
C[0,1] = 0
C[0,2] = 0
for i in range(1,machineNumber):
    for j in range(P):
        C[i, j] = 1
C


{(0, 0): 1,
 (0, 1): 0,
 (0, 2): 0,
 (1, 0): 1,
 (1, 1): 1,
 (1, 2): 1,
 (2, 0): 1,
 (2, 1): 1,
 (2, 2): 1,
 (3, 0): 1,
 (3, 1): 1,
 (3, 2): 1,
 (4, 0): 1,
 (4, 1): 1,
 (4, 2): 1}

In [9]:
I = []
totalLength = len(df)
for i in range(totalLength):
    temp = []
    maxI = 0
    for j in range(maxProcess):
        columnName = "Process " + str(j+1) + ".1"
        print("Column Name = ", columnName, ", item index = ", i, ", value = ", df[columnName][i])
        if(np.isnan(df[columnName][i])):
            break
        else:
            maxI += 1
    I.append(maxI)
I

Column Name =  Process 1.1 , item index =  0 , value =  2.7
Column Name =  Process 2.1 , item index =  0 , value =  1.0
Column Name =  Process 3.1 , item index =  0 , value =  0.5
Column Name =  Process 4.1 , item index =  0 , value =  nan
Column Name =  Process 1.1 , item index =  1 , value =  1.6
Column Name =  Process 2.1 , item index =  1 , value =  1.4
Column Name =  Process 3.1 , item index =  1 , value =  0.9
Column Name =  Process 4.1 , item index =  1 , value =  nan
Column Name =  Process 1.1 , item index =  2 , value =  1.0
Column Name =  Process 2.1 , item index =  2 , value =  0.9
Column Name =  Process 3.1 , item index =  2 , value =  0.2
Column Name =  Process 4.1 , item index =  2 , value =  0.2
Column Name =  Process 5.1 , item index =  2 , value =  1.4
Column Name =  Process 6.1 , item index =  2 , value =  nan
Column Name =  Process 1.1 , item index =  3 , value =  0.5
Column Name =  Process 2.1 , item index =  3 , value =  0.7
Column Name =  Process 3.1 , item index 

[3, 3, 5, 6, 3, 1, 2, 2, 3, 5, 3]

# Problem 1

In [10]:
import numpy as np

In [11]:
from datetime import datetime, date

date_time_str = "7:30:00"

b = datetime.strptime(date_time_str, '%H:%M:%S').time()


print ("The date is", b)

The date is 07:30:00


In [12]:
S = []
D = []
for index, row in df.iterrows():
    S.append(row['Splitting Timing'])
    D.append(row['Due Time'])
D

[datetime.time(17, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(12, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30),
 datetime.time(17, 30)]

In [13]:
peg1 = Model("peg1")


A = {}
for j in range(J):
    # A.append([])
    for i in range(I[j]):
        # A[i].append([])
        count = 0
        for p in range(P-1):
            if(df["Process "+ str(i+1)][j] == method[p]):
                A[i,j,p] = 1
                count = 1
            else:
                A[i,j,p] = 0
        if(count == 0):
            A[i, j, P-1] = 1
        else:
            A[i, j, P-1] = 0
A

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-31


{(0, 0, 0): 1,
 (0, 0, 1): 0,
 (0, 0, 2): 0,
 (1, 0, 0): 0,
 (1, 0, 1): 1,
 (1, 0, 2): 0,
 (2, 0, 0): 0,
 (2, 0, 1): 0,
 (2, 0, 2): 1,
 (0, 1, 0): 1,
 (0, 1, 1): 0,
 (0, 1, 2): 0,
 (1, 1, 0): 0,
 (1, 1, 1): 1,
 (1, 1, 2): 0,
 (2, 1, 0): 1,
 (2, 1, 1): 0,
 (2, 1, 2): 0,
 (0, 2, 0): 1,
 (0, 2, 1): 0,
 (0, 2, 2): 0,
 (1, 2, 0): 0,
 (1, 2, 1): 1,
 (1, 2, 2): 0,
 (2, 2, 0): 1,
 (2, 2, 1): 0,
 (2, 2, 2): 0,
 (3, 2, 0): 0,
 (3, 2, 1): 1,
 (3, 2, 2): 0,
 (4, 2, 0): 1,
 (4, 2, 1): 0,
 (4, 2, 2): 0,
 (0, 3, 0): 0,
 (0, 3, 1): 0,
 (0, 3, 2): 1,
 (1, 3, 0): 1,
 (1, 3, 1): 0,
 (1, 3, 2): 0,
 (2, 3, 0): 0,
 (2, 3, 1): 1,
 (2, 3, 2): 0,
 (3, 3, 0): 1,
 (3, 3, 1): 0,
 (3, 3, 2): 0,
 (4, 3, 0): 0,
 (4, 3, 1): 1,
 (4, 3, 2): 0,
 (5, 3, 0): 0,
 (5, 3, 1): 0,
 (5, 3, 2): 1,
 (0, 4, 0): 1,
 (0, 4, 1): 0,
 (0, 4, 2): 0,
 (1, 4, 0): 0,
 (1, 4, 1): 1,
 (1, 4, 2): 0,
 (2, 4, 0): 1,
 (2, 4, 1): 0,
 (2, 4, 2): 0,
 (0, 5, 0): 1,
 (0, 5, 1): 0,
 (0, 5, 2): 0,
 (0, 6, 0): 0,
 (0, 6, 1): 1,
 (0, 6, 2): 0,
 (1, 6, 0)

In [14]:
T = {}
for j in range(J):
    # temp = []
    for i in range(I[j]):
        if(np.isnan(df["Process " + str(i+1) + ".1"][j])):
            # temp.append(0)
            T[i, j] = 0
        else:
            # temp.append(j)
            T[i, j] = df["Process " + str(i+1) + ".1"][j]
    # T.append(temp)
T

{(0, 0): 2.7,
 (1, 0): 1.0,
 (2, 0): 0.5,
 (0, 1): 1.6,
 (1, 1): 1.4,
 (2, 1): 0.9,
 (0, 2): 1.0,
 (1, 2): 0.9,
 (2, 2): 0.2,
 (3, 2): 0.2,
 (4, 2): 1.4,
 (0, 3): 0.5,
 (1, 3): 0.7,
 (2, 3): 1.0,
 (3, 3): 0.6,
 (4, 3): 0.3,
 (5, 3): 0.5,
 (0, 4): 0.8,
 (1, 4): 1.0,
 (2, 4): 0.9,
 (0, 5): 2.7,
 (0, 6): 1.4,
 (1, 6): 1.5,
 (0, 7): 1.1,
 (1, 7): 1.1,
 (0, 8): 0.8,
 (1, 8): 1.0,
 (2, 8): 0.8,
 (0, 9): 1.0,
 (1, 9): 0.5,
 (2, 9): 0.7,
 (3, 9): 1.1,
 (4, 9): 1.1,
 (0, 10): 1.1,
 (1, 10): 1.4,
 (2, 10): 1.5}

In [15]:
K = 0
for j in range(J):
  for i in range(I[j]):
      K += T[i,j]

In [16]:
### variables
# add x_ijm
x = {}
# x = []
for j in range(J):
    # x.append([])
    for i in range(I[j]):
        # x[i].append([])
        for m in range(M):
            # x[i][j].append(peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x"+ str(i+1) + str(j+1) + str(m+2)));
             x[i,j,m] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "x"+ str(i+1) + str(j+1) + str(m+2))

# add w_j
w = {}
for j in range(J):
    w[j] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "w"+str(j+1))

# add z_iji'j'
z = {}
for j in range(J):
    # z.append([])
    for i in range(I[j]):
        # z[i].append([])
        for j_ in range(J):
            # z[i][j].append([])
            for i_ in range(I[j_]):
                z[i,j,i_,j_] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "z" + str(i+1) + str(j+1) + str(i_+1) + str(j_+1))

# add y_iji'j'm
y = {}
for j in range(J):
    # y.append([])
    for i in range(I[j]):
        # y[i].append([])
        for j_ in range(J):
            # y[i][j].append([])
            for i_ in range(I[j_]):
                # y[i][j][i_].append([])
                for m in range(M):
                    y[i,j,i_,j_,m] = peg1.addVar(lb = 0, vtype = GRB.BINARY, name = "y" + str(i+1) + str(j+1) + str(i_+1) + str(j_+1) + str(m+2))

# add v_ij
v = {}
for j in range(J):
    # v.append([])
    for i in range(I[j]):
        v[i, j] = peg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "v" + str(i+1) + str(j+1))


In [17]:
### obj
peg1.setObjective(quicksum(w[j] for j in range(J)), GRB.MINIMIZE)

# 1
peg1.addConstrs((((((v[i, j] + T[i,j] - v[i_, j] <= K * z[i, j,i_,j]) for j in range(J-1)) for i in range(I[j]-1)) for j_ in range(j+1, J)) for i_ in range(i+1, I[j_])), "same_job1")

peg1.addConstrs((((((v[i_,j] + T[i,j] - v[i,j] <= K * z[i,j,i_,j]) for j in range(J-1)) for i in range(I[j]-1)) for j_ in range(j+1, J)) for i_ in range(i+1, I[j_])),"same_job2")



{}

In [18]:
# add vx_ijm
vx = {}
for j in range(J):
    for i in range(I[j]):
        for m in range(M):
            vx[i,j,m] = peg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "vx" + str(i+1) + str(j+1)+str(m+2))



In [19]:
# peg1.addConstrs((vx[i,j,m] == v[i,j] * x[i,j,m] for j in range(J) for i in range(I[j]) for m in range(M)), "vx")
peg1.addConstrs((y[i,j,i_,j_,m] == x[i,j,m]*x[i_,j_,m] 
                for j in range(J)
                for i in range(I[j]) 
                for j_ in range(J)
                for i_ in range(I[j_]) 
                for m in range(M)
                if (i != i_ or  j != j_)),
                "y_constraint")

{(0, 0, 0, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 4): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 4): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 4): <gurobi.QConstr Not Yet Added>,


In [20]:
# 1-a
peg1.addConstrs(((v[i,j] + T[i_,j] - v[i_,j]) <= K * (1 - z[i,j,i_,j])
    for j in range(J)
    for i in range(I[j]-1) 
    for i_ in range(i+1, I[j])),"same_job1")

# 1-a
peg1.addConstrs(((v[i_,j] + T[i,j] - v[i,j]) <= K * z[i,j,i_,j]
    for j in range(J)
    for i in range(I[j]-1) 
    for i_ in range(i+1, I[j])),"samejob2")

{(0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0, 3): <gurobi.Constr *Awaiting Model Upd

In [21]:
# 1-b
peg1.addConstrs(((y[i,j,i_,j_,m] * ((v[i,j] + T[i_,j_]) - v[i_,j_])) <= (K * (1 - z[i,j,i_,j_]) )
                for j in range(J)  
                for j_ in range(J)
                 for i in range(I[j])  
                for i_ in range(I[j_])  
                for m in range(M)
                if (i != i_ or  j != j_)),"same_machine1")

# 1-b
peg1.addConstrs((y[i,j,i_,j_,m] * (v[i_,j_] + T[i,j] - v[i,j]) <= K * z[i,j,i_,j_]
    for j in range(J) 
    for i in range(I[j])
    for j_ in range(J)
    for i_ in range(I[j_])
    for m in range(M)
    if (i != i_ or  j != j_)),"same_machine2")

{(0, 0, 0, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 1, 4): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 0, 2, 4): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 0): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 1): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 2): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 3): <gurobi.QConstr Not Yet Added>,
 (0, 0, 1, 1, 4): <gurobi.QConstr Not Yet Added>,


In [22]:
import math

In [23]:
# peg1.addConstrs(((x[i,j,m] == 0) >> y[i,j,i_,j_,m] == 0 for i in range(I-1) 
#     for j in range(J-1)
#     for i_ in range(i+1, I)
#     for j_ in range(j+1, J)
#     for m in range(M)), name='y_constraint1')

# peg1.addConstrs(((x[i,j_,m] == 0) >>  y[i,j,i_,j_,m] == 0 for i in range(I-1) 
#     for j in range(J-1)
#     for i_ in range(i+1, I)
#     for j_ in range(j+1, J)
#     for m in range(M)), name='y_constraint2')


In [24]:
# 2
peg1.addConstrs((quicksum(x[i,j,m] for m in range(M)) == 1 for j in range(J) for i in range(I[j]) ), "each process is completed by exactly one machine")


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,


In [25]:
# 3
peg1.addConstrs((x[i,j,m] * A[i,j,p] <= C[m,p] for j in range(J) for i in range(I[j]) for p in range(P) for m in range(M)), "whether a machine can do to process type")


{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Up

In [26]:
J

11

In [27]:
# 4
peg1.addConstrs((v[i+1,j] - v[i,j] == T[i+1,j] for j in range(J) for i in range(S[j]-1)), "split1-1")

peg1.addConstrs((v[i+1,j] - v[i,j] == T[i+1,j] for j in range(J) for i in range(S[j], I[j]-1)), "split1-2")

peg1.addConstrs((x[i+1,j,m] == x[i,j,m] for j in range(J) for i in range(S[j]-1) for m in range(M)), "split2-1")

peg1.addConstrs((x[i+1,j,m] == x[i,j,m] for j in range(J) for i in range(S[j], I[j]-1) for m in range(M)), "split2-2")    


{(0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2, 3): <gurobi.Constr *Awaiting Model Upd

In [28]:
# (datetime.combine(date.today(), D[0]) - datetime.combine(date.today(), b)).total_seconds()

In [29]:
# 5
peg1.addConstrs(
    (v[I[j]-1,j] - ((datetime.combine(date.today(), D[j]) - datetime.combine(date.today(), b)).total_seconds()/3600) <= K*w[j] 
    for j in range(J)),
    "minimum tardiness")


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [30]:
peg1.addConstrs((v[i+1,j] - v[i,j] >= 0 for j in range(J) for i in range(I[j]-1)), "process order of same job")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (7, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 0): <gurobi.Constr *Awaiting Model Update*>,
 (8, 1): <gurobi.Constr *Awaiting Model Update*>,
 (9, 0): <gurobi.Constr *Awaiting Model Update*>,


In [31]:
peg1.addConstrs((v[0,j]  >= T[0,j] for j in range(J)), "first process completion time")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>}

In [32]:
peg1.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 823 rows, 8183 columns and 947 nonzeros
Model fingerprint: 0xa72e59c7
Model has 18900 quadratic constraints
Variable types: 216 continuous, 7967 integer (7967 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [2e-01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 4e+01]
  QRHS range       [4e+01, 4e+01]
Presolve added 8670 rows and 0 columns
Presolve removed 0 rows and 6128 columns
Presolve time: 0.14s
Presolved: 12617 rows, 2836 columns, 44838 nonzeros
Variable types: 20 continuous, 2816 integer (2816 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 0.000000e+00, 70 iterations, 0.05 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective

In [33]:
for var in peg1.getVars():
  print(var.varName, " = ", var.x)  
print("objective value =", peg1.objVal)
# # peg1.computeIIS()
# # peg1.write("model.ilp")

x112  =  -0.0
x113  =  -0.0
x114  =  -0.0
x115  =  1.0
x116  =  -0.0
x212  =  0.0
x213  =  1.0
x214  =  0.0
x215  =  0.0
x216  =  0.0
x312  =  0.0
x313  =  1.0
x314  =  0.0
x315  =  0.0
x316  =  0.0
x122  =  -0.0
x123  =  -0.0
x124  =  -0.0
x125  =  1.0
x126  =  -0.0
x222  =  0.0
x223  =  1.0
x224  =  0.0
x225  =  0.0
x226  =  0.0
x322  =  0.0
x323  =  1.0
x324  =  0.0
x325  =  0.0
x326  =  0.0
x132  =  1.0
x133  =  -0.0
x134  =  -0.0
x135  =  -0.0
x136  =  -0.0
x232  =  0.0
x233  =  0.0
x234  =  1.0
x235  =  0.0
x236  =  0.0
x332  =  0.0
x333  =  0.0
x334  =  1.0
x335  =  0.0
x336  =  0.0
x432  =  0.0
x433  =  0.0
x434  =  1.0
x435  =  0.0
x436  =  0.0
x532  =  0.0
x533  =  0.0
x534  =  1.0
x535  =  0.0
x536  =  0.0
x142  =  0.0
x143  =  0.0
x144  =  0.0
x145  =  0.0
x146  =  1.0
x242  =  0.0
x243  =  0.0
x244  =  0.0
x245  =  0.0
x246  =  1.0
x342  =  0.0
x343  =  0.0
x344  =  0.0
x345  =  0.0
x346  =  1.0
x442  =  0.0
x443  =  0.0
x444  =  0.0
x445  =  0.0
x446  =  1.0
x542  =  0.0


z14510  =  1.0
z14111  =  -0.0
z14211  =  1.0
z14311  =  1.0
z2411  =  1.0
z2421  =  1.0
z2431  =  1.0
z2412  =  0.0
z2422  =  1.0
z2432  =  1.0
z2413  =  0.0
z2423  =  0.0
z2433  =  1.0
z2443  =  1.0
z2453  =  1.0
z2414  =  0.0
z2424  =  0.0
z2434  =  1.0
z2444  =  1.0
z2454  =  1.0
z2464  =  1.0
z2415  =  -0.0
z2425  =  1.0
z2435  =  1.0
z2416  =  1.0
z2417  =  0.0
z2427  =  1.0
z2418  =  -0.0
z2428  =  1.0
z2419  =  1.0
z2429  =  -0.0
z2439  =  1.0
z24110  =  -0.0
z24210  =  -0.0
z24310  =  1.0
z24410  =  1.0
z24510  =  1.0
z24111  =  -0.0
z24211  =  1.0
z24311  =  1.0
z3411  =  1.0
z3421  =  1.0
z3431  =  1.0
z3412  =  0.0
z3422  =  1.0
z3432  =  1.0
z3413  =  0.0
z3423  =  0.0
z3433  =  0.0
z3443  =  0.0
z3453  =  1.0
z3414  =  0.0
z3424  =  0.0
z3434  =  0.0
z3444  =  1.0
z3454  =  1.0
z3464  =  1.0
z3415  =  -0.0
z3425  =  1.0
z3435  =  1.0
z3416  =  1.0
z3417  =  0.0
z3427  =  1.0
z3418  =  -0.0
z3428  =  -0.0
z3419  =  1.0
z3429  =  -0.0
z3439  =  -0.0
z34110  =  -0.0
z34210  

z210111  =  0.0
z210211  =  0.0
z210311  =  0.0
z31011  =  1.0
z31021  =  1.0
z31031  =  1.0
z31012  =  0.0
z31022  =  1.0
z31032  =  1.0
z31013  =  0.0
z31023  =  0.0
z31033  =  0.0
z31043  =  0.0
z31053  =  0.0
z31014  =  0.0
z31024  =  0.0
z31034  =  1.0
z31044  =  1.0
z31054  =  0.0
z31064  =  0.0
z31015  =  1.0
z31025  =  1.0
z31035  =  1.0
z31016  =  1.0
z31017  =  0.0
z31027  =  1.0
z31018  =  1.0
z31028  =  1.0
z31019  =  1.0
z31029  =  1.0
z31039  =  1.0
z310110  =  0.0
z310210  =  0.0
z310310  =  0.0
z310410  =  1.0
z310510  =  1.0
z310111  =  0.0
z310211  =  0.0
z310311  =  0.0
z41011  =  1.0
z41021  =  1.0
z41031  =  1.0
z41012  =  0.0
z41022  =  0.0
z41032  =  1.0
z41013  =  0.0
z41023  =  0.0
z41033  =  0.0
z41043  =  1.0
z41053  =  1.0
z41014  =  0.0
z41024  =  0.0
z41034  =  0.0
z41044  =  0.0
z41054  =  1.0
z41064  =  1.0
z41015  =  0.0
z41025  =  0.0
z41035  =  0.0
z41016  =  0.0
z41017  =  0.0
z41027  =  1.0
z41018  =  1.0
z41028  =  1.0
z41019  =  0.0
z41029  =  1.0

y315106  =  0.0
y311112  =  0.0
y311113  =  1.0
y311114  =  0.0
y311115  =  0.0
y311116  =  0.0
y312112  =  0.0
y312113  =  1.0
y312114  =  0.0
y312115  =  0.0
y312116  =  0.0
y313112  =  0.0
y313113  =  -0.0
y313114  =  0.0
y313115  =  0.0
y313116  =  0.0
y12112  =  0.0
y12113  =  0.0
y12114  =  0.0
y12115  =  1.0
y12116  =  0.0
y12212  =  0.0
y12213  =  -0.0
y12214  =  0.0
y12215  =  0.0
y12216  =  0.0
y12312  =  0.0
y12313  =  0.0
y12314  =  0.0
y12315  =  0.0
y12316  =  0.0
y12122  =  0.0
y12123  =  0.0
y12124  =  0.0
y12125  =  0.0
y12126  =  0.0
y12222  =  0.0
y12223  =  0.0
y12224  =  0.0
y12225  =  0.0
y12226  =  0.0
y12322  =  0.0
y12323  =  0.0
y12324  =  0.0
y12325  =  0.0
y12326  =  0.0
y12132  =  0.0
y12133  =  0.0
y12134  =  -0.0
y12135  =  -0.0
y12136  =  0.0
y12232  =  0.0
y12233  =  0.0
y12234  =  0.0
y12235  =  0.0
y12236  =  0.0
y12332  =  0.0
y12333  =  0.0
y12334  =  0.0
y12335  =  -0.0
y12336  =  0.0
y12432  =  0.0
y12433  =  0.0
y12434  =  0.0
y12435  =  0.0
y124

y13533  =  0.0
y13534  =  0.0
y13535  =  0.0
y13536  =  0.0
y13142  =  0.0
y13143  =  0.0
y13144  =  0.0
y13145  =  0.0
y13146  =  0.0
y13242  =  0.0
y13243  =  0.0
y13244  =  0.0
y13245  =  0.0
y13246  =  0.0
y13342  =  0.0
y13343  =  -0.0
y13344  =  0.0
y13345  =  0.0
y13346  =  0.0
y13442  =  0.0
y13443  =  0.0
y13444  =  0.0
y13445  =  0.0
y13446  =  0.0
y13542  =  0.0
y13543  =  -0.0
y13544  =  -0.0
y13545  =  0.0
y13546  =  0.0
y13642  =  0.0
y13643  =  0.0
y13644  =  0.0
y13645  =  0.0
y13646  =  0.0
y13152  =  1.0
y13153  =  0.0
y13154  =  0.0
y13155  =  -0.0
y13156  =  -0.0
y13252  =  0.0
y13253  =  -0.0
y13254  =  -0.0
y13255  =  -0.0
y13256  =  -0.0
y13352  =  0.0
y13353  =  0.0
y13354  =  0.0
y13355  =  0.0
y13356  =  0.0
y13162  =  1.0
y13163  =  0.0
y13164  =  -0.0
y13165  =  0.0
y13166  =  0.0
y13172  =  0.0
y13173  =  0.0
y13174  =  -0.0
y13175  =  0.0
y13176  =  -0.0
y13272  =  0.0
y13273  =  -0.0
y13274  =  0.0
y13275  =  -0.0
y13276  =  0.0
y13182  =  0.0
y13183  =  

...

y24292  =  0.0
y24293  =  0.0
y24294  =  0.0
y24295  =  0.0
y24296  =  0.0
y24392  =  0.0
y24393  =  0.0
y24394  =  0.0
y24395  =  0.0
y24396  =  0.0
y241102  =  0.0
y241103  =  0.0
y241104  =  0.0
y241105  =  0.0
y241106  =  0.0
y242102  =  0.0
y242103  =  0.0
y242104  =  0.0
y242105  =  0.0
y242106  =  0.0
y243102  =  0.0
y243103  =  0.0
y243104  =  -0.0
y243105  =  0.0
y243106  =  0.0
y244102  =  0.0
y244103  =  0.0
y244104  =  0.0
y244105  =  0.0
y244106  =  1.0
y245102  =  0.0
y245103  =  0.0
y245104  =  0.0
y245105  =  -0.0
y245106  =  1.0
y241112  =  0.0
y241113  =  0.0
y241114  =  0.0
y241115  =  -0.0
y241116  =  0.0
y242112  =  0.0
y242113  =  0.0
y242114  =  0.0
y242115  =  0.0
y242116  =  0.0
y243112  =  0.0
y243113  =  0.0
y243114  =  0.0
y243115  =  0.0
y243116  =  0.0
y34112  =  0.0
y34113  =  0.0
y34114  =  0.0
y34115  =  0.0
y34116  =  0.0
y34212  =  0.0
y34213  =  0.0
y34214  =  0.0
y34215  =  0.0
y34216  =  0.0
y34312  =  0.0
y34313  =  -0.0
y34314  =  0.0
y34315  =  

y15254  =  0.0
y15255  =  0.0
y15256  =  0.0
y15352  =  0.0
y15353  =  0.0
y15354  =  0.0
y15355  =  0.0
y15356  =  0.0
y15162  =  1.0
y15163  =  0.0
y15164  =  -0.0
y15165  =  -0.0
y15166  =  -0.0
y15172  =  0.0
y15173  =  0.0
y15174  =  -0.0
y15175  =  0.0
y15176  =  -0.0
y15272  =  0.0
y15273  =  0.0
y15274  =  -0.0
y15275  =  -0.0
y15276  =  0.0
y15182  =  0.0
y15183  =  0.0
y15184  =  0.0
y15185  =  0.0
y15186  =  -0.0
y15282  =  0.0
y15283  =  0.0
y15284  =  0.0
y15285  =  0.0
y15286  =  0.0
y15192  =  0.0
y15193  =  -0.0
y15194  =  0.0
y15195  =  0.0
y15196  =  -0.0
y15292  =  0.0
y15293  =  -0.0
y15294  =  0.0
y15295  =  0.0
y15296  =  0.0
y15392  =  0.0
y15393  =  0.0
y15394  =  0.0
y15395  =  -0.0
y15396  =  -0.0
y151102  =  0.0
y151103  =  0.0
y151104  =  0.0
y151105  =  0.0
y151106  =  -0.0
y152102  =  0.0
y152103  =  0.0
y152104  =  -0.0
y152105  =  0.0
y152106  =  0.0
y153102  =  0.0
y153103  =  0.0
y153104  =  0.0
y153105  =  0.0
y153106  =  0.0
y154102  =  0.0
y154103  

y281104  =  1.0
y281105  =  0.0
y281106  =  0.0
y282102  =  0.0
y282103  =  0.0
y282104  =  1.0
y282105  =  0.0
y282106  =  0.0
y283102  =  0.0
y283103  =  0.0
y283104  =  1.0
y283105  =  0.0
y283106  =  0.0
y284102  =  0.0
y284103  =  0.0
y284104  =  0.0
y284105  =  0.0
y284106  =  0.0
y285102  =  0.0
y285103  =  0.0
y285104  =  0.0
y285105  =  0.0
y285106  =  0.0
y281112  =  0.0
y281113  =  0.0
y281114  =  0.0
y281115  =  0.0
y281116  =  0.0
y282112  =  0.0
y282113  =  -0.0
y282114  =  0.0
y282115  =  0.0
y282116  =  0.0
y283112  =  0.0
y283113  =  0.0
y283114  =  0.0
y283115  =  0.0
y283116  =  0.0
y19112  =  0.0
y19113  =  0.0
y19114  =  -0.0
y19115  =  -0.0
y19116  =  0.0
y19212  =  0.0
y19213  =  0.0
y19214  =  0.0
y19215  =  0.0
y19216  =  0.0
y19312  =  0.0
y19313  =  -0.0
y19314  =  0.0
y19315  =  0.0
y19316  =  0.0
y19122  =  0.0
y19123  =  0.0
y19124  =  -0.0
y19125  =  -0.0
y19126  =  -0.0
y19222  =  0.0
y19223  =  0.0
y19224  =  0.0
y19225  =  0.0
y19226  =  0.0
y19322  = 

y110155  =  -0.0
y110156  =  0.0
y110252  =  0.0
y110253  =  0.0
y110254  =  -0.0
y110255  =  0.0
y110256  =  -0.0
y110352  =  0.0
y110353  =  0.0
y110354  =  0.0
y110355  =  0.0
y110356  =  0.0
y110162  =  0.0
y110163  =  0.0
y110164  =  -0.0
y110165  =  0.0
y110166  =  0.0
y110172  =  0.0
y110173  =  0.0
y110174  =  0.0
y110175  =  0.0
y110176  =  0.0
y110272  =  0.0
y110273  =  0.0
y110274  =  0.0
y110275  =  0.0
y110276  =  0.0
y110182  =  0.0
y110183  =  0.0
y110184  =  1.0
y110185  =  0.0
y110186  =  0.0
y110282  =  0.0
y110283  =  0.0
y110284  =  1.0
y110285  =  -0.0
y110286  =  -0.0
y110192  =  0.0
y110193  =  0.0
y110194  =  0.0
y110195  =  0.0
y110196  =  0.0
y110292  =  0.0
y110293  =  0.0
y110294  =  0.0
y110295  =  0.0
y110296  =  0.0
y110392  =  0.0
y110393  =  0.0
y110394  =  0.0
y110395  =  0.0
y110396  =  0.0
y1101102  =  0.0
y1101103  =  0.0
y1101104  =  0.0
y1101105  =  0.0
y1101106  =  0.0
y1102102  =  0.0
y1102103  =  -0.0
y1102104  =  1.0
y1102105  =  -0.0
y110210

y211215  =  0.0
y211216  =  0.0
y211312  =  0.0
y211313  =  1.0
y211314  =  0.0
y211315  =  0.0
y211316  =  0.0
y211122  =  0.0
y211123  =  0.0
y211124  =  0.0
y211125  =  0.0
y211126  =  0.0
y211222  =  0.0
y211223  =  1.0
y211224  =  0.0
y211225  =  0.0
y211226  =  0.0
y211322  =  0.0
y211323  =  1.0
y211324  =  0.0
y211325  =  0.0
y211326  =  0.0
y211132  =  0.0
y211133  =  -0.0
y211134  =  -0.0
y211135  =  -0.0
y211136  =  -0.0
y211232  =  0.0
y211233  =  0.0
y211234  =  -0.0
y211235  =  0.0
y211236  =  0.0
y211332  =  0.0
y211333  =  0.0
y211334  =  0.0
y211335  =  0.0
y211336  =  0.0
y211432  =  0.0
y211433  =  0.0
y211434  =  0.0
y211435  =  0.0
y211436  =  0.0
y211532  =  0.0
y211533  =  -0.0
y211534  =  0.0
y211535  =  0.0
y211536  =  0.0
y211142  =  0.0
y211143  =  0.0
y211144  =  0.0
y211145  =  0.0
y211146  =  0.0
y211242  =  0.0
y211243  =  0.0
y211244  =  0.0
y211245  =  0.0
y211246  =  0.0
y211342  =  0.0
y211343  =  0.0
y211344  =  0.0
y211345  =  0.0
y211346  =  0.0
y2

In [34]:
# peg1.computeIIS()
# peg1.write("model.ilp")

# Problem 2

# Problem 3

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d46e1ceb-56b3-4729-a7f8-5832e58e15c6' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>